# 卷积操作

## Tensorflow卷积操作

### 原始结果错误的定义

In [84]:
import numpy as np
import tensorflow as tf

m = np.array([[[[2,6,7],[5,1,9],[2,-1,-5]]]],dtype=np.float)
# m = np.array([[[[2,5,2],[6,1,-1],[7,9,-5]]]],dtype=np.float)
# print(m)
n = np.array([[[[-1,2],[5,1],[4,6]]]], dtype=np.float)
# n = np.array([[[[-1,5,4],[2,1,6]]]], dtype=np.float)
input_data = tf.Variable(m, dtype=np.float)
filter_data = tf.Variable(n, dtype=np.float)
# p = tf.nn.conv2d(m.T, n, strides=[1,1,1,1], padding="SAME")
p = tf.nn.conv2d(input_data, filter_data, strides=[1,1], padding="SAME")
print("conv result: ", p)

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    sess.run(p)
    print("conv result: ", p.eval())

conv result:  Tensor("Conv2D_59:0", shape=(1, 1, 3, 2), dtype=float32)
conv result:  [[[[ 56.  52.]
   [ 36.  65.]
   [-27. -27.]]]]


### 基于维度的修改
#### 在助教启发下，了解了tensor4个维度的意义：batch, height, width, deep，另外在参考别处代码的时候，发现kenerl的顺序是：shape=[kh,kw,n_in,n_out]，其中n_in=input_op.get_shape()[-1].value


In [104]:
import numpy as np

# m = np.random.rand(1,3,1,3)
n = np.random.rand(1, 2, 1, 3)
# print(m)
# m = np.array([[[[2,6,7]],[[5,1,9]],[[2,-1,-5]]]],dtype=np.float)
m = np.array([[[[2,5,2]],[[6,1,-1]],[[7,9,-5]]]],dtype=np.float)
print("m.shape: ", m.shape)
n = np.array([[[[-1,5,4]],[[2,1,6]]]], dtype=np.float)
print("n.shape: ", n.shape)
# n = np.array([[[[-1,5,4],[2,1,6]]]], dtype=np.float)
input_data = tf.Variable(m, dtype=np.float)
filter_data = tf.Variable(n, dtype=np.float)
# p = tf.nn.conv2d(m.T, n, strides=[1,1,1,1], padding="SAME")
p = tf.nn.conv2d(input_data, filter_data, strides=[1,1], padding="SAME")
print("conv result: ", p)

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    sess.run(p)
    print("conv result: ", p.eval())

m.shape:  (1, 3, 1, 3)
n.shape:  (1, 2, 1, 3)
conv result:  Tensor("Conv2D_67:0", shape=(1, 3, 1, 3), dtype=float32)
conv result:  [[[[ -2.  25.   8.]]

  [[ -6.   5.  -4.]]

  [[ -7.  45. -20.]]]]


### 简单卷积

In [ ]:
import numpy as np

# m = np.random.rand(1,3,1,3)
n = np.random.rand(1, 2, 1, 3)
# print(m)
# m = np.array([[[[2,6,7]],[[5,1,9]],[[2,-1,-5]]]],dtype=np.float)
m = np.array([[[[2,5,2]],[[6,1,-1]],[[7,9,-5]]]],dtype=np.float)
print("m.shape: ", m.shape)
n = np.array([[[[-1,5,4]],[[2,1,6]]]], dtype=np.float)
print("n.shape: ", n.shape)
# n = np.array([[[[-1,5,4],[2,1,6]]]], dtype=np.float)
input_data = tf.Variable(m, dtype=np.float)
filter_data = tf.Variable(n, dtype=np.float)
# p = tf.nn.conv2d(m.T, n, strides=[1,1,1,1], padding="SAME")
p = tf.nn.conv2d(input_data, filter_data, strides=[1,1], padding="SAME")
print("conv result: ", p)

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    sess.run(p)
    print("conv result: ", p.eval())

In [99]:
import numpy as np

m = np.random.rand(1,3,1,1)
n = np.random.rand(1, 1, 2, 1)
print("m:\n",m)
print("n:\n",n)

m:
 [[[[0.10432338 0.35168747 0.78435299]
   [0.06552401 0.96728397 0.07425476]
   [0.52219242 0.12154562 0.77048187]]]]
n:
 [[[[0.74790435 0.14675966 0.33817268]
   [0.41021468 0.02811123 0.72906948]]]]


## pytorch卷积操作

In [15]:
import torch.nn as nn
import torch
 
 
im = torch.randn(1, 1, 5, 5)
# print(im)

c = nn.Conv2d(1, 1, kernel_size=2, stride=2, padding=1)
output = c(im)
 
print(c)
print(output)

Conv2d(1, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
tensor([[[[ 0.2990,  0.5587,  0.6199],
          [ 0.2107,  0.2047,  1.7539],
          [ 0.2545, -0.6801,  1.1945]]]], grad_fn=<MkldnnConvolutionBackward>)


# 实现经典网络

## LE-NET-5

In [1]:
import tensorflow as tf
from inputdata import read_data_sets
mnist = read_data_sets("MNIST_data/", one_hot=True)

#标准差为0.1的正态分布
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#0.1的偏差常数，为了避免死亡节点
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#二维卷积函数
#strides代表卷积模板移动的步长，全是1代表走过所有的值
#padding设为SAME意思是保持输入输出的大小一样，使用全0补充
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#ksize [1, height, width, 1] 第一个和最后一个代表对batches和channel做池化，1代表不池化
#strides [1, stride,stride, 1]意思是步长为2，我们使用的最大池化
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32, [None,784])
y_ = tf.placeholder(tf.float32, [None,10])
#reshape图片成28 * 28 大小，-1代表样本数量不固定，1代表channel
x_image=tf.reshape(x,[-1,28,28,1])

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [2]:
#前面两个5代表卷积核的尺寸，1代表channel，32代表深度
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
p_conv1 = max_pool_2x2(h_conv1) 

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(p_conv1,W_conv2)+b_conv2)
p_conv2 = max_pool_2x2(h_conv2) 

#输入为7 * 7 * 64， 输出为1024的一维向量
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(p_conv2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# softmax
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

# 交叉熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),reduction_indices=[1]))
#参数优化器Adam代替梯度递减，当然都可以使用，这里只是用一个和以前不一样的
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
# 正确的预测结果
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# 计算预测准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
# 训练
# tf.global_variables_initilizer().run()
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
for i in range(2000):
  batch = mnist.train.next_batch(50)
  #每一百次打印一次结果,keep_prob设为1意思是用原数据进行检测，所以保留所有数据
  if(i%100==0):
    train_acuracy = accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0}, session=sess)
    print("step %d, tranning accuracy %g:"%(i,train_acuracy))
  train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5}, session=sess)

eval_val=accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}, session=sess)
#训练结束后，在测试集上进行测试
# print("Model accuracy: ", accuracy.eval(feed.dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))
print("Model accuracy: ", eval_val)

step 0, tranning accuracy 0.1:
step 100, tranning accuracy 0.84:
step 200, tranning accuracy 0.92:
step 300, tranning accuracy 0.96:
step 400, tranning accuracy 0.96:
step 500, tranning accuracy 0.96:
step 600, tranning accuracy 0.96:
step 700, tranning accuracy 0.98:
step 800, tranning accuracy 0.88:
step 900, tranning accuracy 0.98:
step 1000, tranning accuracy 0.96:
step 1100, tranning accuracy 0.92:
step 1200, tranning accuracy 0.98:
step 1300, tranning accuracy 0.94:
step 1400, tranning accuracy 1:
step 1500, tranning accuracy 1:
step 1600, tranning accuracy 0.98:
step 1700, tranning accuracy 0.98:
step 1800, tranning accuracy 1:
step 1900, tranning accuracy 0.98:
Model accuracy:  0.9806
